In [1]:
import re
f = open('dataset/bing_liu/Nokia 6610.txt', 'r')

pattern_title = '\[t\]'
pattern_sentence = '(?<=##).+'
pattern_aspect = '.+(?=##)'
pattern_aspect_anomali = '\[(u|p|s|cc|cs)\]'

sentences = []
review = []
count = 0
for a in f:
    if re.search('##', a):
        sentence = re.findall(pattern_sentence, a)[0]
        aspect = re.findall(pattern_aspect, a)
#         anomali = re.findall(pattern_aspect_anomali, a)
#         if anomali:
#             print(sentence, aspect)
#         else:
#             if len(aspect) > 0:
#                 aspect = aspect[0]
#             else:
#                 aspect = ''
        sentences.append(sentence)

In [2]:
import pandas as pd

df = pd.DataFrame(columns=['review','aspect'])
for r in review:
    df = df.append({'review': r[0], 'aspect': r[1]}, ignore_index=True)
df.to_csv("filtered_dataset")

In [2]:
f = open("SentiCircle/nokia.txt","w+")
for i in sentences:
     f.write(i + "\n")
f.close()

In [52]:
from xml.dom import minidom

doc = minidom.parse("dataset/semeval/ABSA15_Restaurants_Test.xml")

In [53]:
sentences = []

reviews = doc.getElementsByTagName('Review')
for review in reviews:
    for sentence in review.getElementsByTagName('sentence'):
        text = sentence.getElementsByTagName('text')[0].firstChild.data
        sentences.append(text)
        for opinion in sentence.getElementsByTagName('Opinion'):
            category = opinion.getAttribute('category')
            polarity = opinion.getAttribute('polarity')
#             print(category, polarity)

In [54]:
f = open("SentiCircle/restaurant.txt","w+")
for i in sentences:
     f.write(i + "\n")
f.close()